In [ ]:
import sys
sys.path.append("../")

import tensorflow as tf
# fix bug for the correct use of Conv2D layers
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

from helpers import normalize_img
from model_setup import training_fit_loop
from data_loading import Data

### Defining the model

In [ ]:
base_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights="imagenet")

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = tf.keras.layers.Dense(10, activation="softmax",
                               name="output_layer")(x)

model_0 = tf.keras.Model(inputs, outputs)

### Training

In [ ]:
a = training_fit_loop(model=model_0,
                     train_data_name="custom",
                     data_step=500,
                     n=4,
                     data_loading_params=("custom", 12, normalize_img, (224, 224), "data/10_food_classes/", "test"),
                     verbose=1)